In [57]:
import mbuild as mb
import numpy as np
from scipy.spatial.distance import pdist
import gsd.hoomd


class LinearCarbonChain(mb.Compound):
    def __init__(self, n_atoms=10):
        super().__init__()
        for i in range(n_atoms):
            self.add(mb.Particle(name="C", pos=[i, 0, 0]))
            if i > 0:
                self.add_bond((self[-2], self[-1]))

class CarbonChainSystem:
    def __init__(self, n_atoms=10, buffer=1.05):
        if buffer <= 1.0:
            raise ValueError()
        self.n_atoms = n_atoms
        self.buffer = buffer
        self.chain = LinearCarbonChain(n_atoms=self.n_atoms)
        self.system = self._build_system()

    def _build_system(self):
        coords = self.chain.xyz
        eucl_dist = pdist(coords)
        chain_length = np.max(eucl_dist)
        box = mb.Box(lengths=np.array([chain_length] * 3) * self.buffer)

        comp = mb.Compound()
        comp.add(self.chain)
        comp.box = box
        self.chain.translate_to([box.Lx / 2, box.Ly / 2, box.Lz / 2])
        return comp

    def save_gsd(self, filename="carbon_chain.gsd"):
        self.system.save(filename, overwrite=True)

if __name__ == "__main__":
    system = CarbonChainSystem(n_atoms=10, buffer=1.2)
    system.save_gsd("carbon_chain.gsd")
